In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)


In [2]:
class Residual(keras.Model):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = layers.Conv2D(
            num_channels, 
            kernel_size=3, 
            strides=strides, 
            padding='same'
        )
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()
        
        self.conv2 = layers.Conv2D(
            num_channels, 
            kernel_size=3, 
            padding='same'
        )
        self.bn2 = layers.BatchNormalization()
        
        self.conv3 = None
        if use_1x1conv:
            self.conv3 = layers.Conv2D(
                num_channels, 
                kernel_size=1, 
                strides=strides
            )
        
        self.relu2 = layers.ReLU()
        return 
    
    def call(self, X):
        Y = self.relu1(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return self.relu2(Y + X)

In [3]:
blk = Residual(3)
#tensorflow input shpe     (n_images, x_shape, y_shape, channels).
#mxnet.gluon.nn.conv_layers    (batch_size, in_channels, height, width) 
X = tf.random.uniform((4, 6, 6 , 3))
blk(X).shape#TensorShape([4, 6, 6, 3])


TensorShape([4, 6, 6, 3])

In [4]:
blk = Residual(6, use_1x1conv=True, strides=2)
blk(X).shape
#TensorShape([4, 3, 3, 6])


TensorShape([4, 3, 3, 6])

In [5]:
class ResnetBlock(layers.Layer):
    def __init__(self, num_channels, num_residuals, first_block=False, **kwargs):
        super().__init__(**kwargs)
        self.listLayers = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                self.listLayers.append(Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                self.listLayers.append(Residual(num_channels))
        return 
    
    def call(self, X):
        for layer in self.listLayers.layers:
            X = layer(X)
        return X

In [11]:
class ResNet(keras.Model):
    def __init__(self, num_blocks, **kwargs):
        super().__init__(**kwargs)
        self.net = keras.Sequential([
            layers.Conv2D(64, kernel_size=7, strides=2, padding='same'), 
            layers.BatchNormalization(), 
            layers.Activation('relu'), 
            layers.MaxPool2D(pool_size=3, strides=2, padding='same'), 
            
            ResnetBlock(64, num_blocks[0], first_block=True), 
            ResnetBlock(128, num_blocks[1]), 
            ResnetBlock(256, num_blocks[2]), 
            ResnetBlock(512, num_blocks[3]), 
            
            layers.GlobalAvgPool2D(), 
            layers.Dense(10, activation='softmax')
        ])
        return 
    
    def call(self, X):
        return self.net(X)
    
net = ResNet([2, 2, 2, 2])

# the net above is same as:
num_blocks = [2, 2, 2, 2]
net = keras.Sequential([
    layers.Conv2D(64, kernel_size=7, strides=2, padding='same'), 
    layers.BatchNormalization(), 
    layers.Activation('relu'), 
    layers.MaxPool2D(pool_size=3, strides=2, padding='same'), 

    ResnetBlock(64, num_blocks[0], first_block=True), 
    ResnetBlock(128, num_blocks[1]), 
    ResnetBlock(256, num_blocks[2]), 
    ResnetBlock(512, num_blocks[3]), 

    layers.GlobalAvgPool2D(), 
    layers.Dense(10, activation='softmax')
])

In [12]:
input_shape = (1, 224, 224, 1)
X = tf.random.uniform(input_shape)
for blk in net.layers:
    X = blk(X)
    print(blk.name, '\t', X.shape)

net.build(input_shape=input_shape)
print(net.summary())

conv2d_85 	 (1, 112, 112, 64)
batch_normalization_72 	 (1, 112, 112, 64)
activation_4 	 (1, 112, 112, 64)
max_pooling2d_4 	 (1, 56, 56, 64)
resnet_block_16 	 (1, 56, 56, 64)
resnet_block_17 	 (1, 28, 28, 128)
resnet_block_18 	 (1, 14, 14, 256)
resnet_block_19 	 (1, 7, 7, 512)
global_average_pooling2d_4 	 (1, 512)
dense_4 	 (1, 10)
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_85 (Conv2D)           multiple                  3200      
_________________________________________________________________
batch_normalization_72 (Batc multiple                  256       
_________________________________________________________________
activation_4 (Activation)    multiple                  0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 multiple                  0         
___________________________________________________

In [13]:
class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.X_train, self.y_train), (self.X_test, self.y_test) = fashion_mnist.load_data()
        self.X_train = np.expand_dims(self.X_train.astype(np.float32)/255.0,axis=-1)
        self.X_test = np.expand_dims(self.X_test.astype(np.float32)/255.0,axis=-1)
        self.y_train = self.y_train.astype(np.int32)
        self.y_test = self.y_test.astype(np.int32)
        self.num_train, self.num_test = self.X_train.shape[0], self.X_test.shape[0]

    def get_batch_train(self, batch_size):
        index = np.random.randint(0, self.num_train, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_train[index],224,224,)
        return resized_images.numpy(), self.y_train[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, self.num_test, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_test[index],224,224,)
        return resized_images.numpy(), self.y_test[index]

batch_size = 64
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)


x_batch shape: (64, 224, 224, 1) y_batch shape: (64,)


In [14]:
net.compile(
    optimizer=keras.optimizers.Adam(1e-7), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

epochs = 1
num_iter = dataLoader.num_train // batch_size
for ep in range(epochs):
    for n in range(num_iter):
        X_batch, y_batch = dataLoader.get_batch_train(batch_size)
        net.fit(X_batch, y_batch)
        if n % 20 == 0:
            net.save_weights('resnet_weights.h5')
            

Train on 64 samples
64/64 [==============================] - 3s 49ms/sample - loss: 2.7258 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 3.2348 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7539 - accuracy: 0.1562
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 3.0128 - accuracy: 0.1562
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7336 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7501 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.9176 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 3.0133 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 3.0658 - accuracy: 0.0781
Train on 64 sample

64/64 [==============================] - 0s 7ms/sample - loss: 2.9009 - accuracy: 0.0312
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.9232 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7777 - accuracy: 0.1562
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7346 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.9105 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.8047 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7946 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.9222 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7699 - accuracy: 0.0938
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 7ms/sample - loss: 2.6905 - accuracy: 0.1562
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.8116 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7695 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7635 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.8093 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.8804 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.6961 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.8325 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7307 - accuracy: 0.1094
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 7ms/sample - loss: 2.6802 - accuracy: 0.1719
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.6581 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.6536 - accuracy: 0.1719
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.4456 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.5715 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.9307 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.6970 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7949 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.6580 - accuracy: 0.1875
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 7ms/sample - loss: 2.5987 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7044 - accuracy: 0.1562
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.4942 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.4894 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7101 - accuracy: 0.1562
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.7152 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.3920 - accuracy: 0.1562
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.5060 - accuracy: 0.2188
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.6781 - accuracy: 0.0781
Train on 64 samples
64/64 [============

...

64/64 [==============================] - 0s 7ms/sample - loss: 2.0582 - accuracy: 0.3594
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.4596 - accuracy: 0.2031
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 1.9701 - accuracy: 0.3125
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.4542 - accuracy: 0.1719
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.4443 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.1889 - accuracy: 0.2344
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.3041 - accuracy: 0.2969
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.4648 - accuracy: 0.2500
Train on 64 samples
64/64 [==============================] - 1s 9ms/sample - loss: 2.3039 - accuracy: 0.2656
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 7ms/sample - loss: 2.1203 - accuracy: 0.2969
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.4604 - accuracy: 0.2344
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 2.0835 - accuracy: 0.2656
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2352 - accuracy: 0.2500
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.3722 - accuracy: 0.2031
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.1911 - accuracy: 0.2500
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.3632 - accuracy: 0.2500
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.0351 - accuracy: 0.2656
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.3671 - accuracy: 0.1719
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 7ms/sample - loss: 2.1547 - accuracy: 0.2188
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 1.8868 - accuracy: 0.4062
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.0562 - accuracy: 0.3594
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2772 - accuracy: 0.2344
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.1786 - accuracy: 0.2656
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2431 - accuracy: 0.2344
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.1905 - accuracy: 0.2812
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2555 - accuracy: 0.2656
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.0469 - accuracy: 0.3594
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 8ms/sample - loss: 2.0758 - accuracy: 0.3750
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2280 - accuracy: 0.2500
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 1.9984 - accuracy: 0.3906
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 1.9694 - accuracy: 0.3125
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2488 - accuracy: 0.3281
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2206 - accuracy: 0.2969
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 1.9579 - accuracy: 0.3906
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2747 - accuracy: 0.2969
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 1.9353 - accuracy: 0.3594
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 7ms/sample - loss: 2.3138 - accuracy: 0.2188
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.2966 - accuracy: 0.2188
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.1476 - accuracy: 0.3125
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 2.3065 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.1569 - accuracy: 0.2500
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.0612 - accuracy: 0.3281
Train on 64 samples
64/64 [==============================] - 1s 9ms/sample - loss: 2.1997 - accuracy: 0.2344
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.0202 - accuracy: 0.3438
Train on 64 samples
64/64 [==============================] - 0s 7ms/sample - loss: 2.0236 - accuracy: 0.3125
Train on 64 samples
64/64 [============

In [15]:
net.load_weights('resnet_weights.h5')
X_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(X_test, y_test, verbose=2)

2000/2000 - 5s - loss: 2.0683 - accuracy: 0.3055


[2.0683116912841797, 0.3055]